In [1]:
import json
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score
import pickle
from transformers import *
from tqdm import tqdm, trange
from ast import literal_eval

Using TensorFlow backend.


##### Predicting argument structures (single class) and fd components (multi-label)
##### Using Propara for test

In [2]:
print("GPU Available: {}".format(torch.cuda.is_available()))
n_gpu = torch.cuda.device_count()
print("Number of GPU Available: {}".format(n_gpu))
print("GPU: {}".format(torch.cuda.get_device_name(0)))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU Available: True
Number of GPU Available: 4
GPU: Tesla V100-SXM2-32GB


In [3]:
df = pd.read_pickle("../data/fd_vn_physical_with_tokens_and_pooled_output_v3.pkl")
#df['train'] = [1]*len(df)
df.head()

,cc,sentence,syntax,fd,tokens,verb,tokens_pt,tokens_ids,token_wise_output,pooled_output
0,"[{'text': 'cart', 'q_value': 'INTL', 'index': ...",The grocery cart hit against the wall,Sbj V PathP,Physical Force,[],hit,[],[],[],[]
1,"[{'text': 'type', 'q_value': '+MER', 'index': ...",That type of rope coiled easily around the post,Sbj V PathP,Autonomous Place,[],coil,[],[],[],[]
2,"[{'text': 'rope', 'q_value': '+MER', 'index': ...",The rope coiled around the post,Sbj V PathP,Autonomous Place,[],coil,[],[],[],[]
3,"[{'text': 'company', 'q_value': '+MER', 'index...",The company is wedging into new markets,Sbj V PathP,Autonomous Place,[],wedge,[],[],[],[]
4,"[{'text': 'computer', 'q_value': '+MER', 'inde...",The computer connected well to the network,Sbj V PathP,Autonomous Place,[],connect,[],[],[],[]


In [4]:
#df = pd.concat([df, df_propara], axis=0)
#df[df['syntax']=='']

In [5]:
def get_all_image_schemas(df):

    _fd = df.fd.unique()
    all_fd1 = []
    all_fd2 = []
    all_fd3 = []
    for fd in _fd:
        if type(fd)==str:
            all_fd1.append(fd.split()[0].lower())
            all_fd2.append(fd.split()[1].lower())
            if len(fd.split())>2:
                all_fd3.append(fd.split()[2].lower())
    all_fd1 = list(set(all_fd1))
    all_fd2 = list(set(all_fd2))
    all_fd3 = list(set(all_fd3))
    #all_fd3.append("None")
    
    return all_fd1 + all_fd2 + all_fd3 + ['none']
    
all_fd = get_all_image_schemas(df)

In [6]:
def make_image_schema_df(df):
    all_items = []

    all_fd = get_all_image_schemas(df)
    
    for idx, row in df.iterrows(): 
        sentence = row.sentence
        #train = row.train
        verb = row.verb

        #item = {"sentence": sentence, "verb": verb, "train": train, }
        item = {"sentence": sentence, "verb": verb}
        for element in all_fd:
            item[element]=0
            
        if type(row.fd) == str:
            fd1 = row.fd.split()[0].lower()
            fd2 = row.fd.split()[1].lower()
            item[fd1] = 1
            item[fd2] = 1
            if len(row.fd.split())>2:
                fd3 = row.fd.split()[2].lower()
                item[fd3] = 1
            else:
                item['none'] = 1
            all_items.append(item)

    df1 = pd.DataFrame(all_items)
    
    return df1

df_image_schema = make_image_schema_df(df)

In [7]:
def make_syntax_df(df):
    all_items = []

    all_syntax = list(df.syntax.unique())
    
    for idx, row in df.iterrows(): 
        sentence = row.sentence
        #train = row.train
        verb = row.verb

        #item = {"sentence": sentence,  "verb": verb, "train": train}
        item = {"sentence": sentence,  "verb": verb}
        for element in all_syntax:
            item[element] = 0
            
        item[row.syntax] = 1
        all_items.append(item)

    df1 = pd.DataFrame(all_items)
    
    return df1

df_syntax = make_syntax_df(df)

In [8]:
len(df_syntax)

562

In [9]:
cols_syntax = df_syntax.columns
print(cols_syntax)
label_cols_syntax = list(cols_syntax[2:])
num_labels_syntax = len(label_cols_syntax)
print('Label columns: ', label_cols_syntax)

Index(['sentence', 'verb', 'Sbj V PathP', 'Sbj V Obj PathP', 'Sbj V',
       'Sbj V from Obl', 'Sbj V Obj', 'Sbj V Obj from Obl', 'Sbj V ResultP',
       'Sbj V from/by Obl', 'Sbj V Obj ResultP with Obl', 'Sbj V Obj with Obl',
       'Sbj V with Obl', 'Sbj V Obj ResultP', 'Sbj V Obj ResultP for Obl',
       'Sbj V Obj for Obl', 'Sbj V ResultP Obj', 'Sbj V on Obl',
       'Sbj V at Obl', 'Sbj V Refl', 'Sbj V in/with Obl',
       'Sbj V Obj in/with Obl', 'Sbj V Obj Obj', 'Sbj V with/of Obl',
       'Sbj V Obj LocP with Obl', 'Sbj V Obj LocP', 'Sbj V from/out of Obl',
       'Sbj V Obj from/out of Obl for Obl', 'Sbj V Obj from/out of Obl',
       'Sbj V CogObj', 'Sbj V LocP', 'Sbj V Comp', 'Sbj V Obj to Obl',
       'Sbj V Obj PathP with Obl', 'Sbj V Obj of Obl'],
      dtype='object')
Label columns:  ['Sbj V PathP', 'Sbj V Obj PathP', 'Sbj V', 'Sbj V from Obl', 'Sbj V Obj', 'Sbj V Obj from Obl', 'Sbj V ResultP', 'Sbj V from/by Obl', 'Sbj V Obj ResultP with Obl', 'Sbj V Obj with Obl', 'Sb

In [10]:
def get_index(syn):
    return label_cols_syntax.index(syn)


df['label'] = df['syntax'].apply(get_index)
df.tail()

,cc,sentence,syntax,fd,tokens,verb,tokens_pt,tokens_ids,token_wise_output,pooled_output,label
557,"[{'text': 'Carla', 'q_value': 'VOL', 'index': ...",Carla shoveled the walk,Sbj V Obj,Volitional Deprive,[],shovel,[],[],[],[],4
558,"[{'text': 'Carla', 'q_value': 'VOL', 'index': ...",Carla shoveled the walk clean,Sbj V Obj ResultP,Volitional Deprive,[],shovel,[],[],[],[],11
559,"[{'text': 'Brian', 'q_value': 'VOL', 'index': ...",Brian wiped the counter,Sbj V Obj,Volitional Deprive,[],wipe,[],[],[],[],4
560,"[{'text': 'doctor', 'q_value': 'VOL', 'index':...",The doctor cured Pat of pneumonia,Sbj V Obj of Obl,Volitional Deprive,[],cure,[],[],[],[],32
561,"[{'text': 'Doug', 'q_value': 'VOL', 'index': 1...",Doug cleaned the table of dishes,Sbj V Obj of Obl,Volitional Deprive,[],clean,[],[],[],[],32


In [11]:
cols_image_schema = df_image_schema.columns
label_cols_image_schema = list(cols_image_schema[2:])
num_labels_image_schema = len(label_cols_image_schema)
print('Label columns: ', label_cols_image_schema)

Label columns:  ['volitional', 'autonomous', 'physical', 'mutual', 'instrument', 'self-volitional', 'cause', 'manipulate', 'dynamictexture', 'location', 'form', 'force', 'place', 'motion', 'create', 'internal', 'remove', 'deprive', 'provide', 'attend', 'cos', 'affect', 'xpr', 'purp', 'none']


In [12]:
#df_image_schema.drop(['experience'], axis=1, inplace=True)

df_image_schema['one_hot_labels'] = list(df_image_schema[label_cols_image_schema].values)
df_image_schema.head()

,sentence,verb,volitional,autonomous,physical,mutual,instrument,self-volitional,cause,manipulate,...,remove,deprive,provide,attend,cos,affect,xpr,purp,none,one_hot_labels
0,The grocery cart hit against the wall,hit,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
1,That type of rope coiled easily around the post,coil,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,The rope coiled around the post,coil,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,The company is wedging into new markets,wedge,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,The computer connected well to the network,connect,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [13]:
df_image_schema.iloc[[548]]['one_hot_labels'].to_list()[0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1])

In [14]:
df_image_schema.iloc[[552]]['one_hot_labels'].to_list()[0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1])

In [15]:
df_image_schema.iloc[[1]]['one_hot_labels'].to_list()[0]

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1])

In [16]:
labels_image_schema = list(df_image_schema.one_hot_labels.values)
sentences_image_schema = list(df_image_schema.sentence.values)


labels_syntax = list(df.label.values)
sentences_syntax = list(df.sentence.values)


In [17]:
# Identifying indices of 'one_hot_labels' entries that only occur once - this will allow us to stratify split our training data later
label_counts_image_schema = df_image_schema.one_hot_labels.astype(str).value_counts()
print(label_counts_image_schema)
one_freq_image_schema = label_counts_image_schema[label_counts_image_schema==1].keys()
one_freq_idxs_image_schema = sorted(list(df_image_schema[df_image_schema.one_hot_labels.astype(str).isin(one_freq_image_schema)].index), reverse=True)
print('df image schema label indices with only one instance: ', one_freq_idxs_image_schema)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1]    76
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1]    51
[1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1]    50
[0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]    37
[0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1]    33
[1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1]    32
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1]    27
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1]    22
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1]    20
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1]    20
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1]    18
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1]    15
[0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1]    14
[1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1]    12
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0]    11
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1]    11
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1]    11
[1 0 0 0 0 0 0

In [18]:
label_counts_syntax = df.label.astype(str).value_counts()
print(label_counts_syntax)
one_freq_syntax = label_counts_syntax[label_counts_syntax==1].keys()
one_freq_idxs_syntax = sorted(list(df[df.label.astype(str).isin(one_freq_syntax)].index), reverse=True)
print('df syntax label indices with only one instance: ', one_freq_idxs_syntax)

4     140
2     100
1      80
0      50
11     43
28     39
9      18
6      12
8      10
21     10
13      9
19      8
20      5
15      4
3       4
17      4
26      3
27      3
10      2
23      2
25      2
32      2
18      2
30      1
12      1
14      1
22      1
29      1
24      1
16      1
5       1
31      1
7       1
Name: label, dtype: int64
df syntax label indices with only one instance:  [531, 379, 345, 285, 259, 188, 137, 112, 80, 49]


In [19]:
pretrained_weights = 'roberta-large'

max_length = 30
tokenizer = RobertaTokenizer.from_pretrained(pretrained_weights, do_lower_case=False)
encodings = tokenizer.batch_encode_plus(sentences_syntax,max_length=max_length,pad_to_max_length=True,return_token_type_ids=True) # tokenizer's encoding method
print('tokenizer outputs: ', encodings.keys())

tokenizer outputs:  dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [20]:
input_ids = encodings['input_ids'].copy() # tokenized and encoded sentences
token_type_ids = encodings['token_type_ids'].copy() # token type ids
attention_masks = encodings['attention_mask'].copy() # attention masks

In [21]:
# Gathering single instance inputs to force into the training set after stratified split
one_freq_input_ids_image_schema = [input_ids.pop(i) for i in one_freq_idxs_image_schema]
one_freq_token_types_image_schema  = [token_type_ids.pop(i) for i in one_freq_idxs_image_schema]
one_freq_attention_masks_image_schema  = [attention_masks.pop(i) for i in one_freq_idxs_image_schema]
one_freq_labels_image_schema  = [labels_image_schema.pop(i) for i in one_freq_idxs_image_schema]

In [22]:
input_ids = encodings['input_ids'].copy() # tokenized and encoded sentences
token_type_ids = encodings['token_type_ids'].copy() # token type ids
attention_masks = encodings['attention_mask'].copy() # attention masks

In [23]:
one_freq_input_ids_syntax = [input_ids.pop(i) for i in one_freq_idxs_syntax]
one_freq_token_types_syntax = [token_type_ids.pop(i) for i in one_freq_idxs_syntax]
one_freq_attention_masks_syntax = [attention_masks.pop(i) for i in one_freq_idxs_syntax]
one_freq_labels_syntax = [labels_syntax.pop(i) for i in one_freq_idxs_syntax]

In [24]:
# ??: I don't know if this is the best way to do it: combining one freq labels from image schema and syntax
one_freq_input_ids = one_freq_input_ids_syntax + one_freq_input_ids_image_schema
one_freq_token_types = one_freq_token_types_syntax + one_freq_token_types_image_schema
one_freq_attention_masks = one_freq_attention_masks_syntax + one_freq_attention_masks_image_schema
one_freq_labels = one_freq_labels_syntax + one_freq_labels_image_schema

In [25]:
len(one_freq_input_ids)

24

In [26]:
# Use train_test_split to split our data into train and validation sets

# not sure why examples 548 and 552 don't work with image schemas

train_inputs, validation_inputs, train_labels, validation_labels, train_token_types, validation_token_types, train_masks, validation_masks = \
    train_test_split(input_ids, 
                     labels_syntax, 
                     token_type_ids,
                     attention_masks, 
                     random_state=2020, 
                     test_size=0.20, 
                     stratify = labels_syntax)


In [27]:
# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
train_token_types = torch.tensor(train_token_types)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)
validation_token_types = torch.tensor(validation_token_types)

In [28]:
len(train_labels)

441

In [29]:
# Select a batch size for training. For fine-tuning with XLNet, the authors recommend a batch size of 32, 48, or 128. We will use 32 here to avoid memory issues.
batch_size = 16

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels, train_token_types)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels, validation_token_types)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [30]:
num_labels_syntax

33

In [31]:
model = RobertaForSequenceClassification.from_pretrained(pretrained_weights, num_labels=num_labels_syntax)

#model = RobertaForSequenceClassification.from_pretrained('roberta-large-mnli', num_labels=3)

#model = RobertaForSequenceClassification.from_pretrained('/nfs/research/regan/bin/roberta_models/', num_labels=num_labels)

model.cuda()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm

In [32]:
# setting custom optimization parameters. You may implement a scheduler here as well.
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [33]:
#optimizer = AdamW(optimizer_grouped_parameters,lr=2e-5,correct_bias=True)
optimizer = AdamW(model.parameters(),lr=2e-5)  # Default optimization

# Number of training epochs (authors recommend between 2 and 4)
epochs = 6

num_warmup_steps = batch_size
num_total_steps = batch_size * epochs
#scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_total_steps)  # PyTorch scheduler

In [34]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [35]:
# Store our loss and accuracy for plotting
train_loss_set = []


# Tracking predictions and labels for confusion matrix
y_pred = []
y_true = []

# trange is a tqdm wrapper around the normal python range
for epoch in trange(epochs, desc="Epoch"):
    
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    outputs = model(b_input_ids, token_type_ids=b_token_types, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    logits = outputs[1]
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      logits = output[0]
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    
    if epoch == epochs-1:
        y_pred.extend(np.argmax(logits, axis=1).flatten())
        y_true.extend(label_ids.flatten())

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Train loss: 2.809430088315691


Epoch:  17%|█▋        | 1/6 [00:09<00:47,  9.49s/it]

Validation Accuracy: 0.3059523809523809
Train loss: 2.149220730577196


Epoch:  33%|███▎      | 2/6 [00:18<00:37,  9.38s/it]

...

Validation Accuracy: 0.8023809523809524
Train loss: 0.5542919875255653


Epoch:  83%|████████▎ | 5/6 [00:46<00:09,  9.21s/it]

Validation Accuracy: 0.8470238095238096
Train loss: 0.4345279469021729


Epoch: 100%|██████████| 6/6 [00:55<00:00,  9.19s/it]

Validation Accuracy: 0.7851190476190476


In [36]:
def integrate_propara_data_training():
    
    df_propara = pd.read_pickle("~/src/coling2020-code/data/propara_fd_image_schema_v3.pkl")
    
    df_propara['fd'] = df_propara.apply (lambda row: (row.fd1+" "+row.fd2+" "+row.fd3), axis=1)
     
    return df_propara

df_propara = integrate_propara_data_training()